In [1]:
import json

with open("../jobs/job_postings_example.json") as f:
    data = json.load(f)

In [3]:
type(data)

list

### Firecrawl Tests

In [ ]:
import os
import dotenv
from langchain_community.document_loaders.firecrawl import FireCrawlLoader
from google.adk.tools.langchain_tool import LangchainTool
from google.adk import Agent, Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

dotenv.load_dotenv()

loader = FireCrawlLoader(
    url = "https://firecrawl.dev",
    mode = "scrape",
)
# Ensure TAVILY_API_KEY is set in your environment
if not os.getenv("FIRECRAWL_API_KEY"):
    print("Warning: FIRECRAWL_API_KEY environment variable not set.")

APP_NAME = "scrape_app"
USER_ID = "1234"
SESSION_ID = "session1234"

# docs = []
# docs_lazy = loader.lazy_load()

# # async variant:
# # docs_lazy = await loader.alazy_load()

# for doc in docs_lazy:
#     docs.append(doc)
# print(docs[0].page_content[:100])
# print(docs[0].metadata)

adk_firecrawl_tool = LangchainTool(tool=loader)

# Define the ADK agent, including the wrapped tool
my_agent = Agent(
    name="langchain_tool_agent",
    model="gemini-2.5-flash-preview-04-17",
    description="Agent to answer questions using Firecrawl.",
    instruction="I can answer your questions by searching the internet. Just ask me anything!",
    tools=[adk_firecrawl_tool]
)

session_service = InMemorySessionService()
session = session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=my_agent, app_name=APP_NAME, session_service=session_service)


# Agent Interaction
def call_agent(query):
    content = types.Content(role='user', parts=[types.Part(text=query)])
    events = runner.run(user_id=USER_ID, session_id=SESSION_ID, new_message=content)

    for event in events:
        if event.is_final_response():
            final_response = event.content.parts[0].text
            print("Agent Response: ", final_response)

call_agent("Scrape the latest news from firecrawl.dev")


True

### Word File page count

In [22]:
from docx import Document

# read the document
doc = Document("data/SajjalaSreekarReddy_CV.docx")

In [7]:
# get the text from the document as one string
text = "\n".join([p.text for p in doc.paragraphs if p.text.strip() != ""])

In [16]:
num_pages = sum(p.contains_page_break for p in doc.paragraphs) + 1
print(f"Number of pages: {num_pages}")

Number of pages: 1


In [ ]:
# modify the text in the document (preserving formatting)
for p in doc.paragraphs:
    for run in p.runs:
        if "Sreekar Reddy Sajjala" in run.text:
            run.text = run.text.replace(
                "Sreekar Reddy Sajjala",
                "Sreekar Reddy Sajjala (Modified)"
            )
            break

# # save the modified document
# doc.save("data/SajjalaSreekarReddy_CV_modified.docx")